# Практическая работа 2. Источники данных. Извлечение данных 
# упр1 Вариант 1

ФИО: Алексеев Константин Владимирович

Группа: БД-231м

## Задание 2.2.  Выполнить парсинг веб-страниц с помощью Beautiful Soup и Pandas открытых источников данных

[Источник данных - Сайт Мосбиржи](https://www.moex.com/)

### Парсинг

In [84]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from bs4 import BeautifulSoup as bso

In [24]:
url = 'https://www.moex.com/ru/marketdata/#/mode=groups&group=12&collection=72&boardgroup=9&data_type=current&category=main'

In [25]:
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [26]:
mysoup = bso(response.text, 'html.parser')

In [27]:
print(response)

<Response [200]>


In [28]:
print(f"Content type: \n\t {response.headers['content-type']}")

Content type: 
	 text/html; charset=utf-8


In [29]:
mysoup = bso(response.text, 'html.parser')

In [30]:
print(mysoup.prettify()[:1000])

<!DOCTYPE html>
<html class="ru" lang="ru">
 <head>
  <!--w1-->
  <title>
   Московская Биржа | Ход/Итоги торгов
  </title>
  <!-- Google Tag Manager -->
  <script>
   (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-NHLBNV7');
  </script>
  <!-- End Google Tag Manager -->
  <meta content="56618e65ad1aec37" name="yandex-verification"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="673c9d708859738e" name="yandex-verification"/>
  <meta content="86ae31544df457ff" name="yandex-verification"/>
  <meta content="UzyoygMTAUWLFgbbUqyBicWL7AF2hFPyzV0yM2H2FLs" name="google-site-verification"/>
  <meta content="width=d


In [31]:
print(mysoup.title)

<title>
	Московская Биржа | Ход/Итоги торгов
</title>


In [32]:
mysoup


<!DOCTYPE html>

<html class="ru" lang="ru">
<head>
<!--w1-->
<title>
	Московская Биржа | Ход/Итоги торгов
</title><!-- Google Tag Manager -->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-NHLBNV7');</script>
<!-- End Google Tag Manager -->
<meta content="56618e65ad1aec37" name="yandex-verification"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="673c9d708859738e" name="yandex-verification"/><meta content="86ae31544df457ff" name="yandex-verification"/><meta content="UzyoygMTAUWLFgbbUqyBicWL7AF2hFPyzV0yM2H2FLs" name="google-site-verification"/>
<meta content="width=device-width, initial-scale=1" id="viewport" 

In [33]:
mysoup.table

In [34]:
table_html = mysoup.table.prettify()
print(table_html[:2000])

AttributeError: 'NoneType' object has no attribute 'prettify'

Простыми способами спарсить сайт мосбиржи не получается. На самом деле это можно было понять посмотрев на ссылку, а также на сам сайт (если изменить страницу таблицы, то ничего не изменится в ссылке, а значит там зашити скрипты, которые это сами делают, что делает парсинг гораздо сложнее).

Поэтому я стал смотреть каким образом люди парсят мосбиржи. Нашел [статью](https://habr.com/ru/articles/781006/) на хабре. Попробовал спарсить данные по статье.

In [56]:
import xml.etree.ElementTree as ET 

import pandas as pd 
import requests  

url = "https://iss.moex.com/iss/index.xml"  # url источника данных XML - глобальные справочники ISS

response = requests.get(url)  

root = ET.fromstring(response.content)  

dfs = {} 

for data in root.findall("./data"):  
    id = data.attrib["id"]
    columns = [
        c.attrib["name"] for c in data.findall("./metadata/columns/column")
    ]  # список колонок
    df = pd.DataFrame(columns=columns)  
    rows = data.findall("./rows/row") 

    for row in rows:
        data = dict(zip(columns, [row.attrib.get(c) for c in columns])) 
        df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)  

    dfs[id] = df 



In [62]:
dfs['boards'].head(3)

,id,board_group_id,engine_id,market_id,boardid,board_title,is_traded,has_candles,is_primary
0,177,57,1,1,TQIF,Т+: Паи - безадрес.,1,1,1
1,178,57,1,1,TQTF,Т+: ETF - безадрес.,1,1,1
2,129,57,1,1,TQBR,Т+: Акции и ДР - безадрес.,1,1,1


In [58]:
for k, v in dfs.items():
    print(k)
    print(len(v))

engines
10
markets
44
boards
390
boardgroups
260
durations
7
securitytypes
45
securitygroups
18
securitycollections
103


In [61]:
dfs['markets'].head(3)

,id,trade_engine_id,trade_engine_name,trade_engine_title,market_name,market_title,market_id,marketplace,is_otc,has_history_files,has_history_trades_files,has_trades,has_history,has_candles,has_orderbook,has_tradingsession,has_extra_yields,has_delay
0,5,1,stock,Фондовый рынок и рынок депозитов,index,Индексы фондового рынка,5,INDICES,0,1,0,1,1,1,0,1,0,0
1,1,1,stock,Фондовый рынок и рынок депозитов,shares,Рынок акций,1,MXSE,0,1,1,1,1,1,1,1,0,1
2,2,1,stock,Фондовый рынок и рынок депозитов,bonds,Рынок облигаций,2,MXSE,0,1,1,1,1,1,1,1,1,1


Данные получились достаточно странные, поэтому я решил что мосбиржу одолеть мне не удастся.

В итоге решил попробовать более простой вариант - [Котировки корпоративных облигаций ](https://smart-lab.ru/q/bonds/). 

In [63]:
url = 'https://smart-lab.ru/q/bonds/order_by_val_to_day/desc/page1/'

In [64]:
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [65]:
print(response)

<Response [200]>


In [66]:
print(f"Content type: \n\t {response.headers['content-type']}")


Content type: 
	 text/html; charset=utf-8


In [67]:
mysoup = bso(response.text, 'html.parser')


In [68]:
print(mysoup.prettify()[:1000])

<!DOCTYPE html>
<html lang="ru">
 <head>
  <title>
   котировки корпоративных облигаций, Московская биржа
  </title>
  <meta charset="utf-8"/>
  <meta content="telephone=no" name="format-detection"/>
  <link as="font" href="/templates/skin/smart-lab-2023/fonts/Tahoma-short-Bold.woff" rel="preload"/>
  <link as="font" href="/templates/skin/smart-lab-2023/fonts/Tahoma-short-Bold.woff2" rel="preload"/>
  <link as="font" href="/templates/skin/smart-lab-2023/fonts/Tahoma-short.woff" rel="preload"/>
  <link as="font" href="/templates/skin/smart-lab-2023/fonts/Tahoma-short.woff2" rel="preload"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <link href="https://smart-lab.ru/q/bonds/" rel="canonical"/>
  <!-- Global Site Tag (gtag.js) - Google Analytics -->
  <script defer="" src="https://www.googletagmanager.com/gtag/js?id=UA-16537214-3">
  </script>
  <script>
   window.dataLayer = window.dataLayer || [];
		function gtag(){dataLayer.push(arguments);}
		gtag('js',

In [69]:
print(mysoup.title)

<title>котировки корпоративных облигаций, Московская биржа</title>


In [70]:
table_html = mysoup.table.prettify()
print(table_html[:2000])

<table class="_hidden">
 <tbody>
  <tr>
   <th class="trades-table__n">
    №
   </th>
   <th class="trades-table__name trades-table__name--center">
    <a href="/q/bonds/order_by_short_name/asc/">
     Имя
    </a>
   </th>
   <th class="trades-table__chartrow chartrow">
   </th>
   <th>
    <a href="/q/bonds/order_by_mat_years/desc/">
     Лет до
     <br/>
     погаш.
    </a>
   </th>
   <th>
    <a href="/q/bonds/order_by_yield/desc/" title="Доходность облигации к погашению при текущей рыночной цене">
     Доходн
    </a>
   </th>
   <th title="Доходность годового купона относительно номинала">
    <a href="/q/bonds/order_by_year_yield/desc/">
     Год.куп.
     <br/>
     дох.
    </a>
   </th>
   <th title="Доходность купона относительно текущей рыночной цены облигации">
    <a href="/q/bonds/order_by_yield_last/desc/">
     Куп.дох.
     <br/>
     посл.
    </a>
   </th>
   <th>
    <a href="/q/bonds/order_by_sm_rating/desc/">
     Рейтинг
    </a>
   </th>
   <th class="trade

In [71]:
columns = []  
data = []     

for th in mysoup.table.find_all('th'):
    columns.append(th.text) 

for tr in mysoup.table.find_all('tr')[1:]: 
    row = [td.text for td in tr.find_all('td')] 
    data.append(row)

df = pd.DataFrame(data, columns=columns)

df.head()

,№,Имя,,Лет допогаш.,Доходн,Год.куп.дох.,Куп.дох.посл.,Рейтинг,"Объем, млн руб","Купон, руб","Частота,раз в год","НКД, руб","Дюр-я, лет",Цена,Дата купона,Размещение,Погашение,Оферта,,
0,1,ВЭБP-40,-,8.0,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t0.0%\n\t\t\t\t...,\n\t\t\t\t\t\t\t0.0%\n\t\t\t\t\t\t,\n\t\t\t\t\t\t\t-\t\t\t\t\t\t,-,798.1,0,4,1.4,-,99.70,09.07.24,09.04.24,30.03.32,30.03.32,+,+
1,2,РусГидрБП9,,1.8,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t14.8%\n\t\t\t\...,\n\t\t\t\t\t\t\t9.2%\n\t\t\t\t\t\t,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t10.0%\n\t\t\t\...,AAA,558.3,45.87,2,18.2,1.67,92.11,31.07.24,01.02.23,28.01.26,-,+,+
2,3,Магнит2Р04,,0.1,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t16.9%\n\t\t\t\...,\n\t\t\t\t\t\t\t7.0%\n\t\t\t\t\t\t,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t7.1%\n\t\t\t\t...,AAA,112.9,35.15,2,26.1,0.13,98.85,29.05.24,02.06.21,29.05.24,-,+,+
3,4,НЛМКЗО26-Д,,2.1,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t4.7%\n\t\t\t\t...,\n\t\t\t\t\t\t\t5.2%\n\t\t\t\t\t\t,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t5.2%\n\t\t\t\t...,-,102.1,23.5,2,1 606.2,-,99.95,30.05.24,18.12.23,30.05.26,-,+,+
4,5,ТМК ЗО2027,,2.8,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t5.8%\n\t\t\t\t...,\n\t\t\t\t\t\t\t4.3%\n\t\t\t\t\t\t,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t4.5%\n\t\t\t\t...,A+,100.9,21.5,2,668.4,2.69,96.00,12.08.24,10.01.24,12.02.27,-,+,+


In [74]:
# Аналогичным образом спарсим оставшиеся 17 страниц

for i in range(1, 19):
    url = f'https://smart-lab.ru/q/bonds/order_by_val_to_day/desc/page{i}/'
    response = requests.get(url)
    if response.status_code == 200:
        soup = bso(response.content, 'html.parser')
        mysoup = bso(response.text, 'html.parser')
        columns = []  
        data = []
        for th in mysoup.table.find_all('th'):
            columns.append(th.text) 

        for tr in mysoup.table.find_all('tr')[1:]:  
            row = [td.text for td in tr.find_all('td')]  
            data.append(row)

        df = pd.DataFrame(data, columns=columns) if i == 1 else pd.concat([df, pd.DataFrame(data, columns=columns)])

# Подготовка данных

In [76]:
df.head(3)

,№,Имя,,Лет допогаш.,Доходн,Год.куп.дох.,Куп.дох.посл.,Рейтинг,"Объем, млн руб","Купон, руб","Частота,раз в год","НКД, руб","Дюр-я, лет",Цена,Дата купона,Размещение,Погашение,Оферта,,
0,1,ВЭБP-40,-,8.0,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t0.0%\n\t\t\t\t...,\n\t\t\t\t\t\t\t0.0%\n\t\t\t\t\t\t,\n\t\t\t\t\t\t\t-\t\t\t\t\t\t,-,798.1,0,4,1.4,-,99.70,09.07.24,09.04.24,30.03.32,30.03.32,+,+
1,2,РусГидрБП9,,1.8,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t14.8%\n\t\t\t\...,\n\t\t\t\t\t\t\t9.2%\n\t\t\t\t\t\t,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t10.0%\n\t\t\t\...,AAA,558.3,45.87,2,18.2,1.67,92.11,31.07.24,01.02.23,28.01.26,-,+,+
2,3,Магнит2Р04,,0.1,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t16.9%\n\t\t\t\...,\n\t\t\t\t\t\t\t7.0%\n\t\t\t\t\t\t,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t7.1%\n\t\t\t\t...,AAA,112.9,35.15,2,26.1,0.13,98.85,29.05.24,02.06.21,29.05.24,-,+,+


Как можно заметить, данные в достаточно плохом виде. Для анализа точно не подходят. Необходимо их преобразовать и почистить.

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1774 entries, 0 to 73
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   №                  1774 non-null   object
 1   Имя                1774 non-null   object
 2                      1774 non-null   object
 3   Лет допогаш.       1774 non-null   object
 4   Доходн             1774 non-null   object
 5   Год.куп.дох.       1774 non-null   object
 6   Куп.дох.посл.      1774 non-null   object
 7   Рейтинг            1774 non-null   object
 8   Объем, млн руб     1774 non-null   object
 9   Купон, руб         1774 non-null   object
 10  Частота,раз в год  1774 non-null   object
 11  НКД, руб           1774 non-null   object
 12  Дюр-я, лет         1774 non-null   object
 13  Цена               1774 non-null   object
 14  Дата купона        1774 non-null   object
 15  Размещение         1774 non-null   object
 16  Погашение          1774 non-null   object
 1

Откуда то появились колонки без имени (2, 18, 19) да и № тоже не сильно нужен

In [80]:
df = df.drop(df.columns[[0, 2, 18, 19]], axis=1)

In [85]:
# заменим прочерки на реальные наны
df = df.replace({"-": np.nan})

In [86]:
df

,Имя,Лет допогаш.,Доходн,Год.куп.дох.,Куп.дох.посл.,Рейтинг,"Объем, млн руб","Купон, руб","Частота,раз в год","НКД, руб","Дюр-я, лет",Цена,Дата купона,Размещение,Погашение,Оферта
0,ВЭБP-40,8.0,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t0.0%\n\t\t\t\t...,\n\t\t\t\t\t\t\t0.0%\n\t\t\t\t\t\t,\n\t\t\t\t\t\t\t-\t\t\t\t\t\t,NaN,798.1,0,4,1.4,NaN,99.70,09.07.24,09.04.24,30.03.32,30.03.32
1,РусГидрБП9,1.8,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t14.8%\n\t\t\t\...,\n\t\t\t\t\t\t\t9.2%\n\t\t\t\t\t\t,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t10.0%\n\t\t\t\...,AAA,558.3,45.87,2,18.2,1.67,92.11,31.07.24,01.02.23,28.01.26,NaN
2,Магнит2Р04,0.1,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t16.9%\n\t\t\t\...,\n\t\t\t\t\t\t\t7.0%\n\t\t\t\t\t\t,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t7.1%\n\t\t\t\t...,AAA,112.9,35.15,2,26.1,0.13,98.85,29.05.24,02.06.21,29.05.24,NaN
3,НЛМКЗО26-Д,2.1,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t4.7%\n\t\t\t\t...,\n\t\t\t\t\t\t\t5.2%\n\t\t\t\t\t\t,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t5.2%\n\t\t\t\t...,NaN,102.1,23.5,2,1 606.2,NaN,99.95,30.05.24,18.12.23,30.05.26,NaN
4,ТМК ЗО2027,2.8,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t5.8%\n\t\t\t\t...,\n\t\t\t\t\t\t\t4.3%\n\t\t\t\t\t\t,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t4.5%\n\t\t\t\t...,A+,100.9,21.5,2,668.4,2.69,96.00,12.08.24,10.01.24,12.02.27,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,BCS12/26-2,2.7,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t0.0%\n\t\t\t\t...,\n\t\t\t\t\t\t\t0.0%\n\t\t\t\t\t\t,\n\t\t\t\t\t\t\t-\t\t\t\t\t\t,A,,0,2,NaN,NaN,,01.07.24,21.09.21,30.12.26,NaN
70,PGIL-28,4.5,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t0.0%\n\t\t\t\t...,\n\t\t\t\t\t\t\t3.2%\n\t\t\t\t\t\t,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t3.7%\n\t\t\t\t...,NaN,,16.25,2,1 498.0,NaN,88.00,14.04.24,14.10.21,14.10.28,NaN
71,BCS11/24-8,0.6,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t0.0%\n\t\t\t\t...,\n\t\t\t\t\t\t\t0.0%\n\t\t\t\t\t\t,\n\t\t\t\t\t\t\t-\t\t\t\t\t\t,A,,10.03,1.0,NaN,NaN,,29.11.24,29.11.21,29.11.24,NaN
72,BCS06/25-A,1.1,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t0.0%\n\t\t\t\t...,\n\t\t\t\t\t\t\t0.0%\n\t\t\t\t\t\t,\n\t\t\t\t\t\t\t-\t\t\t\t\t\t,A,,0,1.0,NaN,NaN,,01.12.24,01.12.21,01.06.25,NaN


![Архитектура решения](./resources/image_2.png)